## 数据增强

In [4]:
import os
import random
from PIL import Image

def place_images_on_large_canvas(images, canvas_size=(1280, 1280), min_gap=2, max_gap=5):
    # 初始化一个空的画布
    canvas_width, canvas_height = canvas_size
    canvas = Image.new('RGB', canvas_size, (255, 255, 255))  # 创建一个白色背景的画布
    x_offset, y_offset = 0, 0  # 当前插入小图的坐标
    current_row_height = 0  # 当前行的最大高度，用于决定y方向的偏移

    # 结果图列表
    all_canvases = [canvas]  # 用来存储大图列表

    # 遍历所有小图
    for image in images:
        image_width, image_height = image.size

        # 随机生成横向和纵向间距
        gap_x = random.randint(min_gap, max_gap)  # x方向的间距
        gap_y = random.randint(min_gap, max_gap)  # y方向的间距

        # 判断当前小图是否能够放入当前的画布
        if x_offset + image_width + gap_x > canvas_width:  # 换行
            x_offset = 0
            y_offset += current_row_height + gap_y  # y方向加上当前行的最大高度和间距
            current_row_height = 0  # 重置当前行的最大高度

            # 检查新的一行是否能放得下
            if y_offset + image_height + gap_y > canvas_height:  # 如果当前大图放不下
                new_canvas = Image.new('RGB', canvas_size, (255, 255, 255))  # 创建新画布
                all_canvases.append(new_canvas)  # 添加新大图
                canvas = new_canvas  # 更新当前画布
                y_offset = 0  # 重置y坐标

        # 如果当前图放不下，跳过
        if y_offset + image_height + gap_y > canvas_height:
            continue

        # 将小图粘贴到当前画布上
        canvas.paste(image, (x_offset, y_offset))

        # 更新坐标，考虑横向和纵向的间距
        x_offset += image_width + gap_x  # 更新x坐标，考虑横向间距
        current_row_height = max(current_row_height, image_height)  # 更新当前行的最大高度

    return all_canvases

# 示例使用
if __name__ == "__main__":
    # 假设images是包含所有小图的列表
    source_images_dir = "./source"
    images = [Image.open(os.path.join(source_images_dir,file_path)) for file_path in os.listdir(source_images_dir)]

    canvases = place_images_on_large_canvas(images)

    # 保存所有生成的图像
    for idx, canvas in enumerate(canvases):
        canvas.save(f"output/output_{idx}.png")

